In [132]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [133]:
from classes.grid import Grid

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()
test = open('test.txt', 'r').read()

input = puzzle

# Part 1

In [134]:
class Garden(Grid):
    def __init__(self, grid):
        super().__init__(grid)

        # The below determines the unique regions of the plot. Regions have a number suffix in order
        # to uniquely identify them if there are multiple separate regions of the same label

        # Set of tiles we still need to examine, starts as every plot in the garden
        to_explore = set([(row,col) for row in range(self.num_rows) for col in range(self.num_cols)])
        
        # Keep track of the regions we come across
        self.regions = {}
        self.region_symbol_count = {}

        # Have we explored every plot in the garden?
        while to_explore:
            # Grab a plot we haven't explored yet and identify it
            explore_next = to_explore.pop()
            this_region_tiles = set([explore_next])

            # Make a record of this new region
            cur_region = self.grid[explore_next[0]][explore_next[1]]
            if (cur_region+'1') not in self.regions:
                self.region_symbol_count[cur_region] = 1
                self.regions[cur_region+'1'] = [explore_next]
            else:
                self.region_symbol_count[cur_region] += 1
                self.regions[cur_region+str(self.region_symbol_count[cur_region])] = [explore_next]

            # Have we seen every plot in this particular region?
            while this_region_tiles:
                explore_next = this_region_tiles.pop()
                #print(f'now exploring {explore_next}')
                # Look at neighbouring tiles
                for dir in ['up','down','left','right']:
                    next_row, next_col, next_val = self.get_relative(explore_next[0],explore_next[1],dir)
                    #print(next_row,next_col,next_val)
                    # Are any the same region?
                    if next_val == cur_region and ((next_row,next_col) not in self.regions[cur_region+str(self.region_symbol_count[cur_region])]):
                        # If we see another neighbouring plot in the same region, add it to the next tiles to explore
                        this_region_tiles.add((next_row,next_col))
                        self.regions[cur_region+str(self.region_symbol_count[cur_region])] += [(next_row,next_col)]
                #print(f'removing {explore_next}')
                #print(f'tiles in this region left to check: {this_region_tiles}\n')
                if explore_next in to_explore:
                    to_explore.remove(explore_next)
    
    def get_perimeter(self, region):
        '''
        Returns the perimeter of a given region (in self.regions format!)
        '''
        region_plots = self.regions[region]

        perimeter = 0
        for plot in region_plots:
            for dir in ['up','down','left','right']:
                _, _, adj_plot_val = self.get_relative(plot[0],plot[1],dir)
                if adj_plot_val != region[0]:
                    perimeter += 1
        
        return perimeter

In [135]:
garden_plots_map = input.split('\n')
garden_plots = Garden(garden_plots_map)
#garden_plots.print_grid()

total_price = 0
for region in garden_plots.regions:
    area = len(garden_plots.regions[region])
    perimeter = garden_plots.get_perimeter(region)
    total_price += area*perimeter
total_price


1452678

# Part 2

In [136]:
class Garden(Grid):
    def __init__(self, grid):
        super().__init__(grid)

        # The below determines the unique regions of the plot. Regions have a number suffix in order
        # to uniquely identify them if there are multiple separate regions of the same label

        # Set of tiles we still need to examine, starts as every plot in the garden
        to_explore = set([(row,col) for row in range(self.num_rows) for col in range(self.num_cols)])
        
        # Keep track of the regions we come across
        self.regions = {}
        self.region_symbol_count = {}

        # Have we explored every plot in the garden?
        while to_explore:
            # Grab a plot we haven't explored yet and identify it
            explore_next = to_explore.pop()
            this_region_tiles = set([explore_next])

            # Make a record of this new region
            cur_region = self.grid[explore_next[0]][explore_next[1]]
            if (cur_region+'1') not in self.regions:
                self.region_symbol_count[cur_region] = 1
                self.regions[cur_region+'1'] = [explore_next]
            else:
                self.region_symbol_count[cur_region] += 1
                self.regions[cur_region+str(self.region_symbol_count[cur_region])] = [explore_next]

            # Have we seen every plot in this particular region?
            while this_region_tiles:
                explore_next = this_region_tiles.pop()
                #print(f'now exploring {explore_next}')
                # Look at neighbouring tiles
                for dir in ['up','down','left','right']:
                    next_row, next_col, next_val = self.get_relative(explore_next[0],explore_next[1],dir)
                    #print(next_row,next_col,next_val)
                    # Are any the same region?
                    if next_val == cur_region and ((next_row,next_col) not in self.regions[cur_region+str(self.region_symbol_count[cur_region])]):
                        # If we see another neighbouring plot in the same region, add it to the next tiles to explore
                        this_region_tiles.add((next_row,next_col))
                        self.regions[cur_region+str(self.region_symbol_count[cur_region])] += [(next_row,next_col)]
                #print(f'removing {explore_next}')
                #print(f'tiles in this region left to check: {this_region_tiles}\n')
                if explore_next in to_explore:
                    to_explore.remove(explore_next)
    
    def get_perimeter(self, region):
        '''
        Returns the perimeter of a given region (in self.regions format!)
        '''
        region_plots = self.regions[region]

        perimeter = 0
        for plot in region_plots:
            for dir in ['up','down','left','right']:
                _, _, adj_plot_val = self.get_relative(plot[0],plot[1],dir)
                if adj_plot_val != region[0]:
                    perimeter += 1
        
        return perimeter
    
    def get_number_of_sides(self, region):
        '''
        Returns the number of sides of a given region (in self.regions format!)
        '''
        region_plots = self.regions[region]

        # Create a new, padded grid so that there is a . between each row and column
        # A padded grid is needed to counter the diagonal edge cases, e.g.
        #
        # AABB
        # BBAA
        #

        padded_grid = Grid([['.']*(self.num_cols*2 + 1) for _ in range(self.num_rows*2 + 1)])

        for row in range(self.num_rows):
            for col in range(self.num_cols):
                padded_grid.grid[row*2+1][col*2+1] = self.grid[row][col]
        
        #padded_grid.print_grid()
        
        # Place fences for this region
        for row in range(self.num_rows*2+1):
            for col in range(self.num_cols*2+1):
                for dir in ['up','down']:
                    adj_row, adj_col, adj_region = padded_grid.get_relative(row,col,dir)
                    adj_row2, adj_col2, adj_region2 = padded_grid.get_relative(row,col,dir,step=2)
                    if padded_grid.grid[row][col] == region[0] and adj_region2 != region[0] and ((row-1)//2,(col-1)//2) in self.regions[region]:
                        padded_grid.grid[adj_row][adj_col] = '-'
                for dir in ['left','right']:
                    adj_row, adj_col, adj_region = padded_grid.get_relative(row,col,dir)
                    adj_row2, adj_col2, adj_region2 = padded_grid.get_relative(row,col,dir,step=2)
                    if padded_grid.grid[row][col] == region[0] and adj_region2 != region[0] and ((row-1)//2,(col-1)//2) in self.regions[region]:
                        padded_grid.grid[adj_row][adj_col] = '|'
        
        # Loop again, and join up the fence fragments. Loop needs to be done afterwards so 
        # we don't get the diagonal-edge case causing issues

        # This time we only loop through the inner grid so not to go out of bounds (exclude padded perimeter)
        for row in range(self.num_rows*2-1):
            for col in range(self.num_cols*2-1):
                adjusted_row = row+1
                adjusted_col = col+1
                _,_,above = padded_grid.get_relative(adjusted_row,adjusted_col,'up')
                _,_,below = padded_grid.get_relative(adjusted_row,adjusted_col,'down')
                _,_,right = padded_grid.get_relative(adjusted_row,adjusted_col,'right')
                _,_,left = padded_grid.get_relative(adjusted_row,adjusted_col,'left')

                _,_,next_h = padded_grid.get_relative(adjusted_row,adjusted_col,'right',step=2)
                _,_,next_v = padded_grid.get_relative(adjusted_row,adjusted_col,'down',step=2)

                if next_h == region[0] and padded_grid.grid[adjusted_row][adjusted_col] == region[0] and above == '-':
                    padded_grid.grid[adjusted_row-1][adjusted_col+1] = '-'
                if next_h == region[0] and padded_grid.grid[adjusted_row][adjusted_col] == region[0] and below == '-':
                    padded_grid.grid[adjusted_row+1][adjusted_col+1] = '-'

                if next_v == region[0] and padded_grid.grid[adjusted_row][adjusted_col] == region[0] and left == '|':
                    padded_grid.grid[adjusted_row+1][adjusted_col-1] = '|'
                if next_v == region[0] and padded_grid.grid[adjusted_row][adjusted_col] == region[0] and right == '|':
                    padded_grid.grid[adjusted_row+1][adjusted_col+1] = '|'
        
        #padded_grid.print_grid()

        # Fence placement is now complete. Now need to go through one last time and count up the number of unique fences
        total_fences = 0
        # Count horizontal fences
        for row in range(self.num_rows*2+1):
            last_fence = padded_grid.grid[row][0]
            for col in range(1,self.num_cols*2+1):
                # Skip non-fence rows
                if row % 2 == 1:
                    continue
                cur_fence = padded_grid.grid[row][col]
                if last_fence == '-' and cur_fence != '-':
                    total_fences += 1
                last_fence = cur_fence

        # Count vertical fences
        for col in range(self.num_rows*2+1):
            last_fence = padded_grid.grid[0][col]
            for row in range(1,self.num_cols*2+1):
                # Skip non-fence rows
                if col % 2 == 1:
                    continue
                cur_fence = padded_grid.grid[row][col]
                if last_fence == '|' and cur_fence != '|':
                    total_fences += 1
                last_fence = cur_fence
        
        #print(f'total fences={total_fences}')
        return total_fences
                        
                    





        

In [137]:
from tqdm import tqdm

garden_plots_map = input.split('\n')

garden_plots = Garden(garden_plots_map)

total_price = 0
for region in tqdm(garden_plots.regions):
    area = len(garden_plots.regions[region])
    num_fences = garden_plots.get_number_of_sides(region)
    total_price += area*num_fences
total_price

100%|██████████| 619/619 [03:26<00:00,  3.00it/s]


873584